# Scrap

In [1]:
"""Resource inputs decorator for function argument sourcing.

This module provides a decorator that wraps functions to source specific arguments
through configurable resolution mechanisms, particularly useful for CLI contexts
where string inputs need to be resolved to actual objects/functions.
"""

from functools import partial
from typing import Callable, Dict, Any, Optional, Union
from cw.resolution import resolve_to_function
from i2.wrapper import Ingress, wrap


def _resolve_resource_spec(resource_spec, default_ingress: Callable) -> Callable:
    """Convert a resource specification into a callable resolver.
    
    Args:
        resource_spec: Resource specification - can be:
            - None: use default_ingress
            - Callable: use directly
            - Dict: use as kwargs for partial(default_ingress, **resource_spec)
        default_ingress: Default ingress function to use
        
    Returns:
        Callable that can resolve the input value
        
    Examples:
        >>> from cw.resolution import resolve_to_function
        >>> resolver = _resolve_resource_spec(None, resolve_to_function)
        >>> # resolver is now resolve_to_function
        
        >>> resolver = _resolve_resource_spec(lambda x: x.upper(), resolve_to_function)
        >>> # resolver is the lambda function
        
        >>> resolver = _resolve_resource_spec({'get_func': {'a': 1}.get}, resolve_to_function)
        >>> # resolver is partial(resolve_to_function, get_func={'a': 1}.get)
    """
    if resource_spec is None:
        return default_ingress
    elif callable(resource_spec):
        return resource_spec
    elif isinstance(resource_spec, dict):
        return partial(default_ingress, **resource_spec)
    else:
        raise TypeError(
            f"Resource spec must be None, callable, or dict. Got {type(resource_spec)}"
        )


def _create_resource_kwargs_trans(
    resource_resolvers: Dict[str, Callable]
) -> Callable[[dict], dict]:
    """Create a kwargs transformation function for resource resolution.
    
    Args:
        resource_resolvers: Mapping of parameter names to their resolver functions
        
    Returns:
        Function that transforms kwargs by applying resolvers to specified parameters
    """
    def _kwargs_trans(outer_kwargs: dict) -> dict:
        """Transform kwargs by resolving specified parameters."""
        transformed = {}
        
        for param_name, value in outer_kwargs.items():
            if param_name in resource_resolvers:
                resolver = resource_resolvers[param_name]
                transformed[param_name] = resolver(value)
            else:
                # Parameters not in resource keep their original values
                transformed[param_name] = value
                
        return transformed
    
    return _kwargs_trans


def resource_inputs(
    func: Callable,
    resource: Dict[str, Union[None, Callable, Dict[str, Any]]],
    *,
    default_ingress: Callable = resolve_to_function
) -> Callable:
    """Wrap a function to source specified inputs through configurable resolvers.
    
    This decorator transforms specified function arguments using resolver functions,
    making it particularly useful for CLI contexts where string inputs need to be
    resolved to actual objects/functions.
    
    Args:
        func: The function to wrap
        resource: Dict mapping parameter names to resource specifications:
            - None: use default_ingress (resolve_to_function by default)
            - Callable: use the callable directly as resolver
            - Dict: use as kwargs for partial(default_ingress, **dict)
        default_ingress: Default resolver function (default: resolve_to_function)
        
    Returns:
        Wrapped function with resource resolution applied to specified parameters
        
    Examples:
        >>> def func(apple, banana, carrot):
        ...     return f"{apple=}, {banana=}, {carrot=}"
        
        >>> from cw.resolution import parse_ast_spec
        >>> function_store = {'a': lambda: 1, 'b': lambda: 2}
        >>> 
        >>> wrapped_func = resource_inputs(
        ...     func, 
        ...     resource=dict(
        ...         apple=None,  # Use default resolve_to_function
        ...         carrot=dict(
        ...             func_key_and_kwargs=parse_ast_spec, 
        ...             get_func=function_store.get
        ...         )
        ...     )
        ... )
        
        >>> # Now apple will be resolved via resolve_to_function
        >>> # carrot will be resolved via resolve_to_function with custom params
        >>> # banana remains unchanged (passed through as-is)
    """
    # Build resolver functions for each resourced parameter
    resource_resolvers = {
        param_name: _resolve_resource_spec(resource_spec, default_ingress)
        for param_name, resource_spec in resource.items()
    }
    
    # Create the kwargs transformation function
    kwargs_trans = _create_resource_kwargs_trans(resource_resolvers)
    
    # Create ingress using the Ingress class
    ingress = Ingress(
        inner_sig=func,
        kwargs_trans=kwargs_trans,
        outer_sig=func,  # Keep same signature
        allow_excess=True,
        apply_defaults=True
    )
    
    # Wrap the function using the ingress
    return wrap(func, ingress=ingress)


# Example usage and testing
if __name__ == "__main__":
    def func(apple, banana, carrot):
        """Example function for testing."""
        return f"{apple=}, {banana=}, {carrot=}"

    from cw.resolution import parse_ast_spec

    function_store = {
        'a': lambda: 1,
        'b': lambda: 2,
    }
    
    wrapped_func = resource_inputs(
        func, 
        resource=dict(
            apple=None,  # Use default resolve_to_function
            carrot=dict(
                func_key_and_kwargs=parse_ast_spec, 
                get_func=function_store.get
            )
        )
    )
    
    # Test the wrapped function
    # apple='builtins.len' -> resolve_to_function('builtins.len') -> len function
    # banana='test' -> unchanged (no resource specified)  
    # carrot='a()' -> parsed as AST, resolved via function_store
    
    print("Wrapped function created successfully!")
    print(f"Signature: {wrapped_func.__signature__}")

Wrapped function created successfully!
Signature: (apple, banana, carrot)


In [ ]:
# Test the wrapped function
apple='builtins.len' #-> resolve_to_function('builtins.len') -> len function
banana='test' #-> unchanged (no resource specified)  
carrot='a()' #-> parsed as AST, resolved via function_store

wrapped_func(apple, banana, carrot)  # doctest: +ELLIPSIS
# "apple=<built-in function len>, banana='test', carrot=<function <lambda> ...>"


"apple=<built-in function len>, banana='test', carrot=<function <lambda> at 0x111645fc0>"